<h1><center> Deep Melanoma Classifier</center></h1>


[Melanoma](https://www.mayoclinic.org/diseases-conditions/melanoma/symptoms-causes/syc-20374884) is the most serious type of skin cancer, develops in the cells (melanocytes) that produce melanin, early diagnosis is key for recovery.

This web app outputs the probability of Melanoma in images of skin lesions uploaded by the user. The user can also choose the number of [Test Time Augmentation](https://towardsdatascience.com/test-time-augmentation-tta-and-how-to-perform-it-with-keras-4ac19b67fb4d) (T.T.A.) to get more accurate predictions. 
    
<h3>Directions For Use:</h3>   

    
1. Select the desired # of Test Time Augmentations using the slider.
2. Upload an image of the skin lesion you want to check for Melanoma.

<h3>Outputs</h3>

1. The user, in realtime, will see the test time augmentations being applied and the corresponding model prediction.
2. At the end, The final prediction (avergae of all TTA predictions) is displayed along with the original image uploaded.

<font color='red'>**NOTE:**</font> This is a proof of concept for research purposes ONLY. Always seek professional medical help in a clinical setting for final diagnosis.

In [34]:
## import the req libraries
import numpy as np
import tensorflow as tf
import efficientnet.tfkeras as efn
import ipywidgets as widgets
from PIL import Image
import h5py
import io
from ipywidgets import VBox, Layout
from tensorflow.keras.preprocessing.image import apply_affine_transform
import time
from tensorflow.keras.models import model_from_json

from contextlib import contextmanager
import sys, os

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

In [23]:
## download model weights
! python download_gdrive.py 1Glog8ezbSN-cbZ49UZsL1C9U1AUr_5P0 EfficientNetB5-weights.15.hdf5

In [33]:
## load the model
def getModel():
    model_input = tf.keras.Input(shape=(248, 248, 3), name='imgIn')
    dummy = tf.keras.layers.Lambda(lambda x:x)(model_input)    
    outputs = []    

    ## get the attributes of efn - effNet
    with suppress_stdout():
        constructor = getattr(efn, 'EfficientNetB5');
        ## remove top, use imagenet weights
        x = constructor(include_top=False, weights='imagenet', 
                        input_shape=(248, 248, 3), 
                        pooling='avg')(dummy)

    ## add a dense layer
    x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    outputs.append(x)
    
    ## create model
    model = tf.keras.Model(model_input, outputs, name='effb5')
    return model

model = getModel()


In [25]:
model.load_weights('EfficientNetB5-weights.15.hdf5')

In [26]:
##upload button
btn_upload = widgets.FileUpload(accept ='image/*'
                                , multiple = True
                                , align_items='center'
                                ,layout=Layout(margin='auto'))

In [27]:
## slider for tta
sldr = widgets.IntSlider(value=10
                         , min=5
                         , max=50
                         , step=1
#                         , description='T.T.A.:'
                         , disabled=False
                         , continuous_update=False
                         , orientation='horizontal'
                         , align_items='center'
                         , readout=True
                         , readout_format='d'
                         , layout=Layout(margin='auto'))

In [28]:
out = widgets.Output(layout=Layout(margin='auto'))
lbl_pred = widgets.Label(layout=Layout(margin='auto'))

In [29]:
## the onclick event for upload button
def on_click(change):
    img = widgets.Image(value = btn_upload.data[-1])
    out.clear_output()
    with out: display(img)
    image1 = Image.open(io.BytesIO(btn_upload.data[-1]))
    image = np.array(image1)
    pred_ls =[]
    for i in range(sldr.value):
        ## convert to tensor 
        image = tf.convert_to_tensor(image, tf.float32)
        image = tf.image.resize(image, [256,256])
        image = np.array(image)

        ## translation
        transformation = apply_affine_transform(image
                                                , tx = (np.random.normal(0, 1, 1)[0])*6
                                                , ty = (np.random.normal(0, 1, 1)[0])*6)
        ## rotation 
        transformation = apply_affine_transform(transformation
                                                , theta = 180 * (np.random.normal(0, 1, 1)[0]))
        ## zoom
        transformation = apply_affine_transform(transformation
                                                , zx = 1 + (np.random.normal(0, 1, 1)[0])/6
                                                , zy = 1 + (np.random.normal(0, 1, 1)[0])/6)
        ## shear
        transformation = apply_affine_transform(transformation
                                                , shear = (np.random.normal(0, 1, 1)[0])*1.5
                                                )
        img1 = tf.image.random_flip_left_right(transformation)
        img1 = tf.image.random_hue(img1, 0.01)
        img1 = tf.image.random_saturation(img1, 0.7, 1.3)
        #img1 = tf.image.random_contrast(img1, 0.8, 1.2)
        img1 = tf.image.random_brightness(img1, 0.1)
        img1 = tf.image.random_crop(img1, [250, 250, 3])

        img1 = tf.cast(img1, tf.float32) / 255.0
        input_arr = tf.image.resize(img1, [248,248])
        input_arr = tf.reshape(input_arr, [248,248, 3])
        out.clear_output()
        img2 = Image.fromarray(((np.array(input_arr)*255).astype(np.int8)), 'RGB')
        with out: display(img2)
        input_arr = tf.expand_dims(input_arr, axis=0)
        pred = model.predict(input_arr, batch_size=1)[0][0]
        pred_ls.append(pred)
        lbl_pred.value = f'T.T.A. # {i+1} Prediction :{ str(round(pred,2))}'
        # Wait for 5 seconds
        time.sleep(5)
    
    out.clear_output()
    img = image1.resize((248,248))
    with out: display(img)
    ans = str(round(np.mean(pred_ls),2))    
    lbl_pred.value = f'Final Result - Probability of Melanoma :{ ans}'
    

In [30]:
btn_upload.observe(on_click, names=['data'])

In [31]:
##organize widgets
display(VBox([widgets.Label('Select the # Test Time Augmentations ( T.T.A. ) using the slider',layout=Layout(margin='auto'))
              , sldr
              , widgets.Label('Upload your image',layout=Layout(margin='auto'))
              , btn_upload
              , out
              ,lbl_pred]
             , layout=Layout(border='solid'
                             ,width='50%'
                             ,margin='auto'))
        )

## Notes - 

-  The Inference model in this app is based on the EfficientNet B5 architecture, trained on the [SIIM](https://siim.org/) Melanoma dataset.
- The model achieves SOTA performance of 0.9339 AUROC on the test data utilizing heavy Test Time Augmentation, 55 to be exact.
- Feel free to check out the GitHub repo for the scripts and details to train the model.
- The binder docker is sometimes slow, be patient with the App runtime.

##### PhaleoHealth